# Coroutine

### Objective

* Explore and compare the models programming for asynchronous problems
* Asychronous, use cases and some approaches
* What is the coroutines? How does it works? Comparations units of work
* Coroutine in application

### What is asynchronous?

Follow [Wikipedia](https://en.wikipedia.org/wiki/Asynchrony_(computer_programming)
> Asynchrony, in computer programming, refers to the occurrence of events independent of the main program flow and ways to deal with such events. These may be "outside" events such as the arrival of signals, or actions instigated by a program that take place concurrently with program execution, without the program blocking to wait for results. Asynchronous input/output is an example of the latter cause of asynchrony, and lets programs issue commands to storage or network devices that service these requests while the processor continues executing the program. Doing so provides a degree of parallelism

The under, we have some solutions for asynchronous tasks

![Async models](./assets/images/async-overview.jpg)

Notice, the python thread is the green thread, it is managed by interpreter instead of OS. Thus, python threads aren't really parallelism and I hate it.

We can see that threads and processes owns isolated space memory thus they can independent work with the main process.

Opposite, the event loop maintains tasks, which is shared memory and we must answer the question `How do we organize memory space for independent tasks?`

### When we use event loop, threads and processes?

In computer science, we can classify two classes of task:
* CPU bound
    > In computer science, a computer is CPU-bound (or compute-bound) when the time for it to complete a task is determined principally by the speed of the central processor: processor utilization is high, perhaps at 100% usage for many seconds or minutes. Interrupts generated by peripherals may be processed slowly, or indefinitely delayed.
    
* I/O bound
    > I/O bound refers to a condition in which the time it takes to complete a computation is determined principally by the period spent waiting for input/output operations to be completed. This is the opposite of a task being CPU bound. This circumstance arises when the rate at which data is requested is slower than the rate it is consumed or, in other words, more time is spent requesting data than processing it
    
**Example**:
* using multiprocess (native thread) for I/O bound

    Jason asks Dung that 'What do you do in today?' and Dung reponses that his tasks is pending in testing and he doesn't have task thus he will back to home and 😴.
    
    No, it isn't optimization solution. Instead, Dung should do other tasks until Ms.Quyen said that 'Hey Dung, your tasks failed, fix it' 😥.
    
    Awesome, this is how event loop works for I/O bound tasks
    
* using event loop for CPU bound

    Our team has 5 members and 5 tasks but only Dung takes all tasks then do. Because each task must be committed at the end of the day so Dung do each task for an hour and move on to another task. Thus Dung was sick in the weekend 😷.
    
    No, we have 5 members and why do we push all tasks to Dung? Jason can deal the tasks to other members and end of the day, all tasks are committed but Dung is ok.
    
    This is how multiprocessing (native threads) works.
    
    
Really, event loop useful in the event system and related others. They can be best solution for I/O bound problems  

#### Problems of task in event loop model

We known that we have a context in the function. The context includes variables and they are unlocatted by interpreter after return command executed. 

In I/O problems, we usually have tasks, which consists of many small parts interrupted by commands delay for IO. At the interrupted point, __we need return control for caller (event loop) from callee (task function) and we also need execute this function at the interrupt__, which is return.

==> Solution: coroutine

### What is coroutine?

Donald Knuth says:
> Subroutines are special cases of coroutine


How coroutine works?

![](assets/images/subroutine_coroutine.png)

__Why coroutine useful for event system?__

* is non preemptive scheduling
* can suspend and resume at the anywhere so if data is the stream, they can save memory
* can maintain state
* for I/O bound, coroutine optimize memory and CPU
* they are small

#### Unit of works

|| Process | Native thread | Green thread | Goroutine | Coroutine |
| :-:| :-: | :-: | :-: | :-:| :-: |
|__Memory__| ≤ 8Mb | ≤ Nx2Mb | ≥ 64Kb | ≥ 8Kb | ≥ 0Mb |
|__OS managed__| Yes | Yes | No | No | No |
|__Pre-emptive scheduling__| Yes | Yes | Yes | No | No |
|__Private address space__| Yes | No | No | No | No |
|__Parallel__| Yes | Yes | No | Yes | No |




#### How implement coroutine from scratch?

```c
#include <stdio.h>

int coroutine() {
    static int i = 0, s = 0;
    switch (s) {
        case 0:
            for (i = 0;; ++i) {
                if (!s) s = 1;
                return i;
                case 1:;
            }
    }
}

int main(int argc, char** argv) {
    printf("%d\n", coroutine());     // ?
    printf("%d\n", coroutine());     // ?
    printf("%d\n", coroutine());     // ?
    return 0;
}

```


```python
def coroutine():
    i = 0
    while 1:
        yield i
        i += 1

co = coroutine()
next(co)
next(co)
next(co)
```

#### Can you convert Python code to C?

In [ ]:
def fib():
    a, b = 0, 1
    while True:
        yield a
        a, b = a + b, a

co = fib()
for _ in range(10):
    print(next(co), end=' ')

```c
#include <stdio.h>

int fib() {
    static int i, __resume__ = 0;
    static a = 0, b = 1, c;
    switch (__resume__) {
        case 0:
            for (i = 0;; ++i) {
                if (!__resume__) __resume__ = 1;
                c = a + b;
                b = a;
                a = c;
                return a;
                case 1:;
            }
    }
}

int main() {
    for (int i = 0; i < 10; ++i) {
        printf("%d ", fib());
    }
    return 0;
}
```

In [ ]:
def say():
    yield "C"
    yield "Java"
    yield "Python"
    
co = say()
print(next(co))
print(next(co))
print(next(co))
print(next(co))

```c
#include <stdio.h>

char* say() {
    static int __resume__ = 0;
    switch (__resume__) {
        case 0:
            __resume__ = 1;
            return "C";
        case 1:
            __resume__ = 2;
            return "Java";
        case 2:
            __resume__ = 3;
            return "Python";
        default:
            return NULL;           // GeneratorExit
    }
}

int main() {
    printf("%s\n", say());
    printf("%s\n", say());
    printf("%s\n", say());
    printf("%s\n", say());
    return 0;
}

```

We can see that coroutine need static memory namespace for save context when it suspends and resumes without lost context. In C, static namespace is static variables, they are maintained by OS when function done. In Python, context of function is stored in stack frame. 

Let's think about coroutines are segments of program, not have private memory, not have parallel and very safe

![](assets/images/co-thread.png)

Coroutine increases many bugs of multiprocessing and I think it is best solution for networking tasks because it alive in single process.

In Python, we can define coroutine by use `yield` statement in the function. When we call functions, they return coroutine object instead of final value

In [ ]:
def coro_fn():
    val = yield 'Starting'   # started coroutine and suspend, return control to caller
    print('Consume', val)
    yield 'Hello World'      # produce data
    
co = coro_fn()               # create a new coroutine object
print(co.send(None))         # start coroutine
print(co.send('data'))       # resume coroutine, pass control into coroutine
co.close()                   # close coroutine

Generator is the specical coroutine, they can only produce data without consume data

#### Subcoroutine and `yield from`

Let's see code:

In [ ]:
def g1():
    for i in range(10):
        yield i
        
def g2():
    for i in range(10, 20):
        yield i

In [ ]:
def g():
    for i in g1():
        yield i
    for i in g2():
        yield i

list(g())

We can refactor this code with `yield from`

In [ ]:
def g():
    yield from g1()
    yield from g2()
    
list(g())

#### Build binary tree with `yield from`

In [ ]:
class Node:
    def __init__(self, value=None, left_nodes=None, right_node=None):
        self.left_nodes = left_nodes or []
        self.right_nodes = right_node or []
        self.value = value
        
    def visit(self):
        for node in self.left_nodes:
            yield from node.visit()
        yield self.value
        for node in self.right_nodes:
            yield from node.visit()
            
root = Node(
    0,
    [Node(1, [Node(7), Node(8)]), Node(2, None, [Node(9), Node(10)]), Node(3)],
    [Node(4), Node(5), Node(6)]
)
for value in root.visit():
    print(value, end=' ')

### Coroutine in application


#### 1. Asychronous TCP server

In this case, TCP server is an event system:

* event source: listen socket and connecton sockets

* basically, we have two type of events: EVENT_READ, EVENT_WRITE

* and tasks are coroutines, each task will handles a connection and a event at a time

* we also have an event loop, it is a I/O multiplexing for the file descriptor

In [ ]:
import logging
from sys import stdout
from socket import socket, SOCK_STREAM, AF_INET
from selectors import DefaultSelector, EVENT_READ, EVENT_WRITE


logging.basicConfig(stream=stdout, level=logging.DEBUG)


class Server:
    def __init__(self, host, port, buf_size=64):
        self.addr = (host, port)
        self.poll = DefaultSelector()
        self.m = {}
        self.buf_size = buf_size

    def handle_read(self, sock):  # make isolated environment for each connection
        buffer_size = self.buf_size
        handle_write = self.handle_write

        def _can_read():
            chunks = []
            while 1:
                chunk = sock.recv(buffer_size)
                if chunk.endswith(b'\n\n'):
                    chunks.append(chunk[:-2])
                    break
                else:
                    chunks.append(chunk)
                    yield

            handle_write(sock, b''.join(chunks))

        handler = _can_read()
        self.m[sock] = handler
        self.poll.register(sock, EVENT_READ, handler)

    def handle_write(self, sock, data):     # make isolated environment for each connection
        poll = self.poll
        m = self.m
        buffer_size = self.buf_size

        def _can_write():
            nonlocal data, sock
            start_, end_ = 0, 0
            data = b'Hello ' + data
            len_data = len(data)

            while 1:
                end_ = min(start_ + buffer_size, len_data)
                if start_ >= end_:
                    break
                sock.send(data[start_:end_])
                start_ += buffer_size
                yield    # this is susppend comamnd

            # clear handler of connection and close conection
            poll.unregister(sock)
            del m[sock]
            sock.close()
            del sock

        handler = _can_write()
        m[sock] = handler
        poll.modify(sock, EVENT_WRITE, handler)

    def handle_accept(self, sock):
        while 1:
            s, addr = sock.accept()
            logging.debug(f'Accept the connection from {addr}')
            self.handle_read(s)
            yield

    def mainloop(self):
        try:
            sock = socket(AF_INET, SOCK_STREAM)
            sock.bind(self.addr)
            sock.setblocking(0)
            sock.listen(1024)

            self.m[sock] = self.handle_accept(sock)
            self.poll.register(sock, EVENT_READ, self.m[sock])

            logging.info(f'Server is running at {self.addr}')
            while 1:
                events = self.poll.select()
                for event, _ in events:
                    try:
                        cb = event.data
                        next(cb)
                    except StopIteration:
                        pass
        except Exception as e:
            sock.close()
            self.poll.close()
            raise e

In [ ]:
server = Server('127.0.0.1', 5000)
server.mainloop()

Scheduler and task in real: https://github.com/dabeaz/curio/blob/master/curio/kernel.py#L188

### 2. Streamming system


We can use coroutines build up a data processing system. Basically, the system was separate to logic blocks. They placed in coroutines with owned context. You can see under picture.

![](assets/images/simple-data-processing.png)

Event sources are redis pub/sub, kafka, rabbitmq, user interative and so on.

We can describe many kind of systems if we create specify logic block: filter, conditional, selector, broadcast,...

Example: build statistic IP from access log in the web server

![](assets/images/IP-statistic.png)

In [ ]:
def coroutine(f):
    def decorator(*args, **kwargs):
        co = f(*args, **kwargs)
        co.send(None)   # start coroutine before it's used
        return co
    return decorator
 
@coroutine
def broadcast(targets):
    try:
        while 1:
            data = yield
            for target in targets:
                target.send(data)
    except GeneratorExit:
        for target in targets:
            target.close()
            
@coroutine
def map_(ip, next_):
    try:
        while 1:
            data = yield
            if data.startswith(ip):
                next_.send(ip)
    except GeneratorExit:
        next_.close()
        
@coroutine
def reduce_(on_done):
    m = {}
    try:
        while 1:
            data = yield
            if data not in m:
                m[data] = 1
            else:
                m[data] += 1
    except GeneratorExit:
        on_done(m)

In [ ]:
result = {}
def on_done(r):
    global result
    result = r

reducer = reduce_(on_done)
flow = broadcast([
    map_('83.149.9.216', reducer),
    map_('93.114.45.13', reducer),
    map_('207.241.237.101', reducer),
])

# this is the source data
# We have 10000 lines in this log
%time
with open('assets/files/access.log', 'r') as fp:
    for line in fp.readlines():
        flow.send(line)
    flow.close()

print(result)

#### Improvement

We can wrapper threads (or process or machine) in coroutines, Why not?

Simiplify, I use threads instead of machine

OK, let's redesign above diagram

![](assets/images/IP-Statistic-v2.png)

In above diagram, I move logic into threads and I use queues as a communication channels with threads.

Furthermore, queues is used as buffer if rate of input is greater than rate of output.

In [ ]:
from threading import Thread
from queue import Queue


def coroutine(f):
    def decorator(*args, **kwargs):
        co = f(*args, **kwargs)
        co.send(None)
        return co
    return decorator
 
@coroutine
def broadcast_threaded(targets):
    queue = Queue()
    def _run_target():
        nonlocal queue, targets
        while 1:
            data = queue.get()
            if data is GeneratorExit:
                for target in targets:
                    target.close()
                return
            else:
                for target in targets:
                    target.send(data)
    Thread(target=_run_target).start()
    try:
        while 1:
            data = yield
            queue.put(data)
    except GeneratorExit:
        queue.put(GeneratorExit)
            
@coroutine
def map_threaded(ip, next_):
    queue = Queue()
    def _run_target():
        nonlocal ip, queue
        while 1:
            data = queue.get()
            if data is GeneratorExit:
                next_.close()
                return
            else:
                if data.startswith(ip):
                    while next_.gi_running:
                        pass
                    next_.send(ip)
                    queue.task_done()
    Thread(target=_run_target).start()
    try:
        while 1:
            data = yield
            queue.put(data)
    except GeneratorExit:
        queue.put(GeneratorExit)
        
@coroutine
def reduce_threaded(on_done):
    m = {}
    queue = Queue()
    def _run_target():
        nonlocal queue, m, on_done
        while 1:
            data = queue.get()
            if data is GeneratorExit:
                on_done(m)
                return
            else:
                if data not in m:
                    m[data] = 1
                else:
                    m[data] += 1
    Thread(target=_run_target).start()
    try:
        while 1:
            data = yield
            queue.put(data)
    except GeneratorExit:
        queue.put(GeneratorExit)

In [ ]:
result = {}
def on_done(r):
    global result
    result = r

reducer = reduce_threaded(on_done)
flow = broadcast_threaded([
    map_threaded('83.149.9.216', reducer),
    map_threaded('93.114.45.13', reducer),
    map_threaded('207.241.237.101', reducer),
])

# this is the source data
# We have 10000 lines in this log
%time
with open('assets/files/access.log', 'r') as fp:
    for line in fp.readlines():
        flow.send(line)
    flow.close()

print(result)         # result?

__Notice__

* When we use other unit of work, which not same the flow of coroutines, we should consider coroutine is overloaded. At this time, coroutine is processing data then sources push data to this coroutine. That's dangerous  

* Don't design cyclic models

* Only call `send()` in synchronize flow, I mean only call `send()` method in single thread

### 3. Scheduler for OS

![](./assets/images/os-scheduler.png)

When statement hits a trap, program passes control to OS and OS executes the statements or switch other tasks and passes control to it.

But this model is non preemptive scheduler, I give you this model for explain relationship between `yield` expression and `trap` in OS

In [ ]:
from queue import Queue


class SystemCall:
    __slots__ = ('sched', 'target')

    def handle(self):
        pass


class Task:
    __slots__ = ('id', 'target', 'sendval')
    _id = 0

    def __init__(self, target):
        Task._id += 1
        self.id = Task._id
        self.target = target
        self.sendval = None

    def run(self):
        return self.target.send(self.sendval)


class Scheduler:
    __slots__ = ('taskmap', 'ready')

    def __init__(self):
        self.taskmap = {}
        self.ready = Queue()

    def new(self, target):
        task = Task(target)
        self.taskmap[task.id] = task
        self.schedule(task)
        return task.id

    def mainloop(self):
        while self.taskmap:
            task = self.ready.get()
            try:
                result = task.run()
                if isinstance(result, SystemCall):
                    result.task = task
                    result.sched = self
                    result.handle()
                    continue
            except StopIteration:
                self.exit(task)
            else:
                self.schedule(task)

    def schedule(self, task):
        self.ready.put(task)

    def exit(self, task):
        print('Task %d terminated' % task.id)
        del self.taskmap[task.id]


class GetTid(SystemCall):
    def handle(self):
        self.task.sendval = self.task.id
        self.sched.schedule(self.task)


class NewTask(SystemCall):
    def __init__(self, target):
        self.target = target

    def handle(self):
        tid = self.sched.new(self.target)
        self.task.sendval = tid
        self.sched.schedule(self.task)


class KillTask(SystemCall):
    def __init__(self, tid):
        self.tid = tid

    def handle(self):
        task = self.sched.taskmap.get(self.tid, None)
        if task:
            task.target.close()
            self.task.sendval = True
        else:
            self.task.sendval = False
        self.sched.schedule(self.task)

In [ ]:
def foo():
    tid = yield GetTid()
    print(f'I\'m foo and I am living in {tid} process')
    for i in range(5):
        print(f"Foo {tid} is in {i} step")
        yield

def bar():
    tid = yield GetTid()
    print(f"I'm bar and I'm living in {tid} process")
    yield NewTask(foo())
    for i in range(3):
        print(f"Bar {tid} is in {i} step")
        yield
    yield KillTask(1)

sched = Scheduler()
sched.new(foo())
sched.new(bar())
sched.mainloop()

### Referrences:

* Python documentation https://docs.python.org/3/
* Talk about coroutine https://www.dabeaz.com/coroutines/Coroutines.pdf